In [1]:
import opendp.prelude as dp

In [2]:
dp.enable_features("contrib")

In [3]:
privacy_unit= dp.unit_of(contributions=1)

In [4]:
privacy_loss=dp.loss_of(epsilon=1.0)

In [5]:
# pip install urllib

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement urlib (from versions: none)
ERROR: No matching distribution found for urlib

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import urllib.request
data_url="https://raw.githubusercontent.com/opendp/opendp/sydney/teacher_survey.csv"
with urllib.request.urlopen(data_url) as f:
    data=f.read().decode("utf-8")

context=dp.Context.compositor(
    data=data,
    privacy_unit=privacy_unit,
    privacy_loss=privacy_loss,
    split_evenly_over=3
)

In [7]:
col_names=[
    "names","sex","age","maritalStatus","hasChildren","highestEducationLevel","sourceOfStress","smoker","optimism","lifeSatisfaction","selfEsteem"
]

In [8]:
count_query=(
    context.query().split_dataframe(",",col_names=col_names).select_column("age",str).count().laplace()
)

In [11]:
scale=count_query.param()
print(scale)

3.0000000000000004


In [12]:
accuracy=dp.discrete_laplacian_scale_to_accuracy(scale,alpha=0.05)
accuracy

9.445721638273586

In [13]:
dp_count=count_query.release()
dp_count

7004

In [14]:
dp_count-accuracy,dp_count+accuracy

(6994.554278361727, 7013.445721638273)

In [16]:
mean_query=(
    context.query().split_dataframe(",",col_names=col_names).select_column("age",str).cast_default(float).clamp((17.0,70.0)).resize(size=dp_count,constant=42.0).mean().laplace()
)

In [17]:
mean_query.release()

37.30338921368928

# moving to another dataset

In [7]:
import urllib.request
data_url="https://raw.githubusercontent.com/opendp/opendp/sydney/Sydney_Synth_Pedestrian_Counts_2024_2_3.csv"
with urllib.request.urlopen(data_url) as f:
    data=f.read().decode("utf-8")

In [8]:
privacy_unit=dp.unit_of(contributions=3)

In [9]:
privacy_loss=dp.loss_of(epsilon=1.0)

In [10]:
col_names=["Street","Hour","First Name","Last Name","Age","Gender"]
street_names=["Park Street","Bridge Street", "Market Street"]

In [11]:
context=dp.Context.compositor(
    data=data,
    privacy_unit=privacy_unit,
    privacy_loss=privacy_loss,
    split_evenly_over=10,
)

In [12]:
Bridge_count_query=(
    context.query().split_dataframe(",",col_names=col_names).df_is_equal("Street","Bridge Street").subset_by("Street", keep_columns=["First Name"]).select_column("First Name",str).count().laplace()
)

In [13]:
Bridge_count_query.release()

3842

In [16]:
count_query=(
    context.query().split_dataframe(',',col_names=col_names).select_column("Street",str).count_by_categories(street_names,null_category=False).laplace()
)

In [17]:
count_query.release()

[33786, 3888, 18765]